# Deep Learning Models - ALL FEATURES

In [ ]:
# Similar to the 6 feature models, we decided to use all features to run our model.
# However, we determined that the results were comparable to the 5 features (oddly, as we are dealing with 12+ times the number of variables)

# Because of the extraordinary time spent in compiling the code, we opted to forfeit further analysis of these deep learning models, as they do not have any interpretability nature
# due to its characteristics as a neural network.

In [1]:
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import metrics
from sklearn.metrics import accuracy_score

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GRU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

import matplotlib.pyplot as plt

In [2]:
Final_df = pd.read_csv('./Clean_dataframe/clean_df.csv')

In [3]:
X = Final_df.drop('sea_level', axis = 1)
y = Final_df['sea_level']

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)

In [4]:
sc = StandardScaler()
X_train_sc = sc.fit_transform(X_train)
X_test_sc = sc.transform(X_test)

In [5]:
# construct network
model = Sequential()

# first layer
model.add(Dense(32, input_shape=(67,), activation='relu'))

# second layer
model.add(Dense(32, activation='relu'))

# output layer
model.add(Dense(1))

In [6]:
model.compile(loss='mse', optimizer='adam', metrics=['mae'])

In [7]:
model.fit(X_train_sc, y_train, validation_data=(X_test_sc, y_test), epochs=10, batch_size=512);

Epoch 1/10
139/139 [==============================] - 1s 2ms/step - loss: 7913233.5000 - mae: 2625.1140 - val_loss: 7730767.5000 - val_mae: 2590.0635
Epoch 2/10
139/139 [==============================] - 0s 1ms/step - loss: 6919425.0000 - mae: 2437.4175 - val_loss: 5685321.0000 - val_mae: 2189.4773
Epoch 3/10
139/139 [==============================] - 0s 1ms/step - loss: 3889583.5000 - mae: 1742.9545 - val_loss: 2172343.7500 - val_mae: 1236.8186
Epoch 4/10
139/139 [==============================] - 0s 1ms/step - loss: 1250720.3750 - mae: 901.9398 - val_loss: 743327.2500 - val_mae: 702.8821
Epoch 5/10
139/139 [==============================] - 0s 1ms/step - loss: 612007.8125 - mae: 639.9565 - val_loss: 552224.8125 - val_mae: 610.7736
Epoch 6/10
139/139 [==============================] - 0s 1ms/step - loss: 507669.1562 - mae: 584.6172 - val_loss: 488099.7500 - val_mae: 571.3632
Epoch 7/10
139/139 [==============================] - 0s 1ms/step - loss: 454804.1875 - mae: 550.2527 - val_los

In [8]:
model.predict(X_test_sc)

array([[3099.7908],
       [2437.2837],
       [2940.0833],
       ...,
       [1522.8619],
       [2051.1812],
       [2586.9224]], dtype=float32)

In [9]:
model.evaluate(X_test_sc, y_test, batch_size=512)

47/47 [==============================] - 0s 523us/step - loss: 349389.9062 - mae: 471.9771


[349389.90625, 471.97705078125]

## GridSearch

In [10]:
def model_fn_adv(hidden_neurons = 32, hidden_layers = 5, dropout = 0.5):
    # build framework of model using for loop and if statement
    model = Sequential()
    
    # adding number of layers = specified in calling of function
    for layer in range(hidden_layers):
        if layer == 0:
            model.add(Dense(hidden_neurons, activation = 'relu', input_shape = (67,)))
            model.add(Dropout(dropout))
        else:
            model.add(Dense(hidden_neurons, activation = 'relu'))
            model.add(Dropout(dropout))
    
    # out put layer added here, Identity link function used as 'None'
    model.add(Dense(1, activation = None))

    # using mse as the metric to solve for 
    model.compile(loss = 'mse', optimizer = 'adam')
    
    return model

In [11]:
nn = KerasRegressor(build_fn = model_fn_adv, batch_size = 512, verbose = 0)

In [12]:
nn_params_deep = {
    'hidden_neurons' : [20, 32, 64],
    'hidden_layers'  : [2, 3, 5],
    'dropout'        : [0.1, 0.2, 0.3, 0.4, 0.5],
    'epochs'         : [10, 20, 30, 40, 50]
}

In [13]:
gs = GridSearchCV(nn, param_grid = nn_params_deep, cv = 5)

In [14]:
gs.fit(X_train_sc, y_train)

GridSearchCV(cv=5,
             estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x7fbaa11837c0>,
             param_grid={'dropout': [0.1, 0.2, 0.3, 0.4, 0.5],
                         'epochs': [10, 20, 30, 40, 50],
                         'hidden_layers': [2, 3, 5],
                         'hidden_neurons': [20, 32, 64]})

In [15]:
gs.best_params_

{'dropout': 0.2, 'epochs': 40, 'hidden_layers': 5, 'hidden_neurons': 64}

In [16]:
gs.best_score_

-53445.3125

In [17]:
# Make predictions 
preds = gs.predict(X_test_sc)

In [18]:
# Check the R-squared
metrics.r2_score(y_test, preds)

0.9614068942449125

## Recurrent Neural Networks

In [19]:
features = ['LONGITUDE', 'ELEVATION','TMIN', 'TMAX' ,'PRCP','TAVG']
X = Final_df[features]
y = Final_df['sea_level'].values

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42, shuffle = False)

In [20]:
ss = StandardScaler()
X_train_sc = ss.fit_transform(X_train)
X_test_sc = ss.transform(X_test)

In [21]:
train_sequences = TimeseriesGenerator(X_train_sc, y_train, length = 3, batch_size=64)

In [22]:
test_sequences = TimeseriesGenerator(X_test_sc, y_test, length = 3, batch_size=64)

In [23]:
input_shape = train_sequences[0][0][0].shape

In [24]:
model = Sequential()
model.add(GRU(8, input_shape=input_shape, return_sequences=True)) # True if next layer is RNN
model.add(GRU(8, return_sequences=False)) # False if next layer is Dense

model.add(Dense(4, activation='relu'))

model.add(Dense(1))

In [25]:
model.compile(optimizer='adam', loss='mse')

history = model.fit(train_sequences, validation_data=test_sequences, epochs=50, verbose=0)

In [26]:
# Add early stopping
from tensorflow.keras.callbacks import EarlyStopping

model2 = Sequential()
model2.add(GRU(8, input_shape=input_shape, return_sequences=True)) # True if next layer is RNN
model2.add(GRU(8, return_sequences=False)) # False if next layer is Dense

model2.add(Dense(4, activation='relu'))

model2.add(Dense(1))

model2.compile(optimizer='adam', loss='mse')

es = EarlyStopping(patience = 5)

history2 = model2.fit(train_sequences, validation_data=test_sequences, epochs=100, batch_size = 64, callbacks = [es])

Epoch 1/100
1110/1110 [==============================] - 5s 3ms/step - loss: 8092744.0000 - val_loss: 6932919.0000
Epoch 2/100
1110/1110 [==============================] - 3s 3ms/step - loss: 7531867.0000 - val_loss: 6247217.5000
Epoch 3/100
1110/1110 [==============================] - 3s 3ms/step - loss: 6658524.0000 - val_loss: 5353232.5000
Epoch 4/100
1110/1110 [==============================] - 3s 3ms/step - loss: 5622171.0000 - val_loss: 4377156.5000
Epoch 5/100
1110/1110 [==============================] - 3s 3ms/step - loss: 4537966.0000 - val_loss: 3416674.2500
Epoch 6/100
1110/1110 [==============================] - 3s 3ms/step - loss: 3497204.2500 - val_loss: 2552063.0000
Epoch 7/100
1110/1110 [==============================] - 3s 3ms/step - loss: 2582238.2500 - val_loss: 1857080.7500
Epoch 8/100
1110/1110 [==============================] - 3s 3ms/step - loss: 1858629.0000 - val_loss: 1379957.5000
Epoch 9/100
1110/1110 [==============================] - 3s 3ms/step - loss: 136